### Welcome to the Aryl Chloro/Tosyl (ArClOTs) Selectivity Analysis Project
#### Purpose
The intent of this notebook is two-fold:
1. Investigate the reactivity of various phosphine ligands in a selected reaction to determine an ideal candidate for selective reactions.
2. Serve as an example of machine learning in small-molecule organic chemistry.

#### Running this Code
Via _Binder_ you can execute this entire script in your browser with no manual setup required, just some waiting while things are downloaded in the background. After this initial setup, you can scroll through the entire script, see how things work, and play around with different values to see how things change.

#### Documentation
There is a lot of explanation included around all of the code below to help you work through things line for line, with the idea being that you should be left with questions that you can Google and not anything more foudnational.

##### Step 1: Loading Libraries
All machine learnings algorithms and chemistry tools that we need have (thankfully) already been written by other people, so all we need to do is install them to our notebook here and them use them freely. Here are some brief explanations of what each of these libraries is for:
 - matplotlib: makes plots
 - pandas: interfacing with excel workbooks
 - numpy: adds more datatypes to python, which need in order to store our data
 - sklearn: has all of the machine learning algorithms
 - rdkit: the infamous cheminformatics software package for loading SMILES strings into various computer-readable formats
 - openpyxl: reading and writing excel files
 - ccmblib: handy interface to RDKit to make fingerprint generation easier

In [1]:
%%capture
# the above line just prevents you from getting hundreds of lines of output from the downloaders
!conda install --yes -c rdkit rdkit
!pip install matplotlib pandas numpy sklearn openpyxl
!pip install git+https://github.com/vogt-m/ccbmlib
!conda install -y boost-cpp boost py-boost

##### Step 2: Getting our Data out of Excel
Right now, we have an excel sheet with two important columns: SMILES strings for the ligands, and the Cl:OTs selectivity. In order to get this data into Python, we use pandas like this:

In [2]:
import pandas
data = pandas.read_excel("data/original_paper_data.xlsx", engine='openpyxl')
data

,idx,ligand_name,ligand_smiles,cl_to_ots_selectivity
0,1,PPh3,P(C1=CC=CC=C1)(C2=CC=CC=C2)C3=CC=CC=C3,99.000000
1,2,P(p-OMePh)3,COC(C=C1)=CC=C1P(C2=CC=C(OC)C=C2)C3=CC=C(OC)C=C3,99.000000
2,3,P(p-FPh)3,FC(C=C1)=CC=C1P(C2=CC=C(F)C=C2)C3=CC=C(F)C=C3,99.000000
3,4,PPhMe2,CP(C)C1=CC=CC=C1,0.175439
4,5,PPhEt2,CCP(C1=CC=CC=C1)CC,2.200000
5,6,PPh2Me,CCP(C1=CC=CC=C1)C2=CC=CC=C2,1.000000
6,7,PCy3,P(C1CCCCC1)(C2CCCCC2)C3CCCCC3,1.500000
7,8,P(n-Bu)3,CCCCP(CCCC)CCCC,1.100000
8,9,P(i-bu)3,CC(C)CP(CC(C)C)CC(C)C,2.900000
9,10,Bis(dicyclohexylphosphino)ferrocene,C1=CC=C(C=C1)P([C-]2C=CC=C2)C3=CC=CC=C3.C1=CC=...,99.000000


Now let's get the SMILES strings into a list, as well as the selectivity values.

In [3]:
smiles = data["ligand_smiles"].to_numpy()
smiles

array(['P(C1=CC=CC=C1)(C2=CC=CC=C2)C3=CC=CC=C3',
       'COC(C=C1)=CC=C1P(C2=CC=C(OC)C=C2)C3=CC=C(OC)C=C3',
       'FC(C=C1)=CC=C1P(C2=CC=C(F)C=C2)C3=CC=C(F)C=C3',
       'CP(C)C1=CC=CC=C1', 'CCP(C1=CC=CC=C1)CC',
       'CCP(C1=CC=CC=C1)C2=CC=CC=C2', 'P(C1CCCCC1)(C2CCCCC2)C3CCCCC3',
       'CCCCP(CCCC)CCCC', 'CC(C)CP(CC(C)C)CC(C)C',
       'C1=CC=C(C=C1)P([C-]2C=CC=C2)C3=CC=CC=C3.C1=CC=C(C=C1)P([C-]2C=CC=C2)C3=CC=CC=C3.[Fe+2]',
       'CCP(CC)CC', 'CP(C)C'], dtype=object)

In [4]:
selectivity = data["cl_to_ots_selectivity"].to_numpy()
selectivity

array([99.        , 99.        , 99.        ,  0.1754386 ,  2.2       ,
        1.        ,  1.5       ,  1.1       ,  2.9       , 99.        ,
        1.8       ,  0.15873016])

##### Step 3: Change the SMILES Strings into Descriptors
Your computer needs to have some representation of the molecules in order to draw conclusions about the reactivity; for this, we use rdkit. This package allows us to generate digital representations of the molecules using fingerprints. To do so, we follow this general procedure:
1. import the functions from rdkit that we need to use
2. Make two empty lists to store our molecules and ginerprints.
3. for each of the SMILES strings, generate a molecule
4. for each of the molecules, generate a molecular fingerprint

In [19]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
import numpy as np

molecules = []
fingerprints = []

for smile in smiles:
    molecules.append(Chem.MolFromSmiles(smile))

for molecule in molecules:
    # start by generating the fingerprint as a bit vector, which is a list of 1's and 0's
    temp_fingerprint = AllChem.GetMorganFingerprintAsBitVect(molecule, 2)
    
    # in order to use this later, we need to convert it into an array. to begin, make an empty array
    temp_array = np.array((0,), dtype=np.int8)
    
    # put the fingerprint into the array
    DataStructs.ConvertToNumpyArray(temp_fingerprint, temp_array)
    
    # save this temporary array into the fingerprint array
    fingerprints.append(temp_array)

Take a look at the cell below, which shows what the fingerprints array _actually looks like_. It is a list, and every component of the list is itself a list of 1's and 0's. Each of those sub-lists represents one of the molecules in our dataset, and the computer can interpret the 1's and 0's, as well as their relative placement to one another, as a molecule.

In [20]:
fingerprints

[array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 1, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 1, 0, 0], dtype=int8),
 array([0, 1, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int8)]

### To-Do List
 - add section for fingerprinting with ccbmlib, which does not guarantee uniform-length outputs and will therefore necessitate zero padding. - switch descriptor calculation to PADEL-Py